In [1]:
%matplotlib inline

import os  
os.chdir('../')

In [2]:
ls data/cifar-10-batches-py/

batches.meta  data_batch_2  data_batch_4  readme.html
data_batch_1  data_batch_3  data_batch_5  test_batch


In [3]:
import numpy as np
import pickle
from functools import reduce
from matplotlib import pyplot as plt

import pandas as pd

In [4]:
def load_files(input_dir):
    """Takes in an input training directory and returns 
    
        train_data, train_labels
        test_data, test_labels"""
    training_files = map(lambda x: import_image(''.join([input_dir, '/', 'data_batch_', str(x)])), range(2, 5))
    
    train_data, train_labels = reduce(lambda x, y: (np.append(x[0], y[0], axis=0), np.append(x[1], y[1], axis=0)), training_files)
    test_data, test_labels = import_image('data/cifar-10-batches-py/test_batch')
    
    return train_data, train_labels, test_data, test_labels

In [5]:
def unpickle_and_encode(file):
    with open(file, 'rb') as fo:
        dic = pickle.load(fo, encoding='bytes')
    return {str(k, 'utf-8'): v for k, v in dic.items()}

In [6]:
def import_image(path):
    raw_image_data = unpickle_and_encode(path)
    
    images, labels = raw_image_data['data'], raw_image_data['labels']
    images = images.reshape((images.shape[0], 3, 32, 32)).transpose(0, 2, 3, 1) 
    
    return images, labels

In [7]:
def plot_image(img_object, label = None):
    
    plt.title(label)
    plt.imshow(img_object)

    

def normalize(x):
    """
        argument
            - x: input image data in numpy array [32, 32, 3]
        return
            - normalized x 
    """
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x-np.min(x)) / (np.max(x)-np.min(x))
    return x

In [8]:
train_data, train_labels, test_data, test_labels = load_files('data/cifar-10-batches-py')
label_encodings = unpickle_and_encode('data/cifar-10-batches-py/batches.meta')['label_names']

In [9]:
train_data, test_data = normalize(train_data), normalize(test_data)

In [10]:
import numpy as np 
import torch
import torchvision
import torchvision.transforms as transforms

In [11]:
from torch.autograd import Variable
import torch.nn as nn

import torch.nn.functional as F

class VanillaCNN(torch.nn.Module):
    
    def __init__(self, num_classes=10):
        super(VanillaCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) 
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
    
class VanillaCNN_With_Dropout(torch.nn.Module):
    
    def __init__(self, num_classes=10):
        super(VanillaCNN_With_Dropout, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) 
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [12]:
import torch.optim as optim
from torch.utils.data import TensorDataset


def loss_and_optimizer(net, learning_rate):
    
    loss = torch.nn.CrossEntropyLoss()
    
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, weight_decay=5e-4)
    
    return loss, optimizer

In [13]:
def get_train_loader(batch_size, train_data, train_labels):
    
    dataset = TensorDataset(torch.Tensor(train_data.transpose(0,3,1,2)),
                            torch.LongTensor(train_labels))
    if batch_size is None:
        batch_size = len(train_data)
    
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                        num_workers=2)





In [14]:
def train_model(net, batch_size, n_epochs, learning_rate, train_data, train_labels, test_data, test_labels):
    training_loss, validation_loss, accuracy_list = [], [], []
    train_loader = get_train_loader(batch_size, train_data, train_labels)
    
    test_loader = get_train_loader(None, test_data, test_labels)
    n_batches = len(train_loader)
    loss, optimizer = loss_and_optimizer(net, learning_rate)
    
    for epoch in range(n_epochs):
        running_loss = 0
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            
            
            inputs, labels = data
           # inputs, labels = Variable(inputs), Variable(labels)
            optimizer.zero_grad()
            
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            
            running_loss += loss_size.data.item() * batch_size
            total_train_loss += loss_size.data.item()

                       
        for inputs, labels in test_loader: 
            val_outputs = net(inputs)
            val_loss = loss(val_outputs, labels).data.item()
            accuracy = (torch.max(val_outputs, 1).indices == labels).sum().item()/test_data.shape[0]
            print(val_loss)
            print("Epoch {0}, \n \
                   Train Loss: {1} \n \
                   Validation Loss: {2} \
                   Accuracy: {3}".format(str(epoch), total_train_loss/(train_data.shape[0]/batch_size), val_loss, accuracy))
            
            accuracy_list.append(accuracy)
        validation_loss.append(val_loss)
        training_loss.append(total_train_loss/(train_data.shape[0]/batch_size))
        running_loss = 0

                
    
    return training_loss, validation_loss, accuracy_list, 
                
            
            
        
            
        

In [17]:
import pandas as pd 
new_vanilla = VanillaCNN_With_Dropout()
losses = train_model(new_vanilla, 32, 30, .01, train_data, train_labels, test_data, test_labels)

2.302248954772949
Epoch 0, 
                    Train Loss: 2.3039719701131185 
                    Validation Loss: 2.302248954772949                    Accuracy: 0.1082
2.2999823093414307
Epoch 1, 
                    Train Loss: 2.3026705464680988 
                    Validation Loss: 2.2999823093414307                    Accuracy: 0.1212
2.2822935581207275
Epoch 2, 
                    Train Loss: 2.296788776397705 
                    Validation Loss: 2.2822935581207275                    Accuracy: 0.1312
2.1713039875030518
Epoch 3, 
                    Train Loss: 2.223897725168864 
                    Validation Loss: 2.1713039875030518                    Accuracy: 0.1907
2.00156307220459
Epoch 4, 
                    Train Loss: 2.0711895495096844 
                    Validation Loss: 2.00156307220459                    Accuracy: 0.2714
1.937245488166809
Epoch 5, 
                    Train Loss: 1.9567533711751302 
                    Validation Loss: 1.937245488166809         

KeyboardInterrupt: 

In [18]:
df = pd.DataFrame(data=losses, index=['train_loss','validation_loss', 'accuracy']).T
df[['train_loss', 'validation_loss']].plot(kind='line')
plt.xlabel('epochs')
plt.show()



NameError: name 'losses' is not defined

In [17]:

import torchvision
import torchvision.transforms as transforms
import torchvision.transforms as transforms

In [ ]:
new = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [22]:
ls data/

cifar-10-batches-py/  input_data.tar.gz


In [48]:
test_loader = get_train_loader(None, test_data, test_labels)


TypeError: 'DataLoader' object is not subscriptable

In [100]:
res = new_vanilla(torch.Tensor(test_data.transpose(0,3,1,2)))

In [101]:
train_data.shape

(30000, 32, 32, 3)

In [108]:
(torch.max(res, 1).indices == torch.Tensor(test_labels)).sum().item()

4154